# Notebook for training the YOLOv8 model

In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [3]:
DATA_PATH = "/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/"
TRAINING_IMAGES_PATH = DATA_PATH + "data/train/images/"
TRAINING_LABELS_PATH = DATA_PATH + "data/train/labels/"

In [8]:
# Read images and annotations
images = [TRAINING_IMAGES_PATH + x for x in os.listdir(TRAINING_IMAGES_PATH) if x[-3:] == "jpg"]
annotations = [TRAINING_LABELS_PATH + x for x in os.listdir(TRAINING_LABELS_PATH) if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_labels, val_labels = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
#val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [9]:
train_images[:5]

['/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/train/images/210216-4x20|_3_6_7.jpg',
 '/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/train/images/210216-4x10|_6_2_3.jpg',
 '/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/train/images/210216-4x10|_7_7_4.jpg',
 '/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/train/images/dst4|_0_7_0.jpg',
 '/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/train/images/210216-4x20|_2_4_2.jpg']

# Train the model

In [10]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="config.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

#results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
#path = model.export(format="onnx")  # export the model to ONNX format

⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt...
######################################################################## 100.0%
New https://pypi.org/project/ultralytics/8.0.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.195 🚀 Python-3.11.5 torch-2.0.1 CPU (Intel Core(TM) i5-7360U 2.30GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show

In [11]:
path = model.export(format="onnx")

Ultralytics YOLOv8.0.195 🚀 Python-3.11.5 torch-2.0.1 CPU (Intel Core(TM) i5-7360U 2.30GHz)

PyTorch: starting from 'runs/detect/train8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.9 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.1s, saved as 'runs/detect/train8/weights/best.onnx' (11.7 MB)

Export complete (2.8s)
Results saved to /Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/notebooks/yolo_model/runs/detect/train8/weights
Predict:         yolo predict task=detect model=runs/detect/train8/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train8/weights/best.onnx imgsz=640 data=config.yaml  
Visualize:       https://netron.app


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



# Testing